In [1]:
# inports
import os
from langchain_community.document_loaders import PyPDFLoader,TextLoader, CSVLoader
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from decouple import config


c:\Users\hetvi\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Embedding model
model_name = "BAAI/bge-large-en-v1.5"
model_kwargs = {'device':'cuda'}
encode_kwargs = {'normalize_embeddings': True} 
bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

C:\Users\hetvi\AppData\Local\Temp\ipykernel_13112\1101999149.py:4: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  bge_embeddings = HuggingFaceBgeEmbeddings(


In [10]:
# llm model & prompt
model = "gemini-pro"
model = ChatGoogleGenerativeAI(model=model,google_api_key=config('GOOGLE_API_KEY'),
                            temperature=0.7)

template = """You are a Chatbot that can answer questions from users based on the Context provided. Ensure the responses are accurate and utilize retrieval from the knowledge base.

Context: {context},
Question: {question}

Note: Give your response in Markdown.
"""

prompt = ChatPromptTemplate.from_template(template)

chain = prompt | model


In [4]:
# Loading file
file_path = './AI Intern Assignment.pdf'
file_type = os.path.splitext(file_path)[1]

if file_type == '.pdf':
    loader = PyPDFLoader(file_path)
    
elif file_type == '.csv':
    loader = CSVLoader(file_path, encoding='utf-8')

elif file_type == '.json':
    loader = TextLoader(file_path)

document = loader.load()

# dividing page content into chunks
if not isinstance(document, CSVLoader):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=200)
    document = text_splitter.split_documents(document)

In [5]:
# initialising vector store and retriver
vector_store = FAISS.from_documents(documents=document, embedding=bge_embeddings)
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 5})

In [ ]:
# Asking Question & retriving response
question = "give me Deliverables in points."
context = retriever.invoke(question)

result = chain.invoke({'question': question, 'context': context})
print(result.content)

- Python Code
 - Submit either .ipynb (Google Colab) or .py files with proper comments and documentation
- Sample Questions and Chatbot Responses
 - Save these in a .txt, .pdf, or .xlsx file
- GitHub Repository
 - Push all code, notebooks, and relevant files to the repository
- Optional Streamlit Link (Bonus)
 - If hosted, include the Streamlit application link in your submission email
